In [54]:
import os
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
from shutil import copy2 as copy

sprite_path = 'images/Unit Sprites/archive/Engage'
out_path = 'images/Unit Sprites/Engage'
label_map = { ""  : ["Unique", "Unique"],
              'MN': ["Male","Neutral"], 
              'MB': ["Male","Blue"],
              'MR': ["Male","Red"], 
              'MG': ["Male","Green"], 
              'MY': ["Male","Yellow"], 
              'FN': ["Female", "Neutral"], 
              'FB': ["Female", "Blue"], 
              'FR': ["Female", "Red"], 
              'FG': ["Female", "Green"], 
              'FY': ["Female", "Yellow"],
              'E':  ['Unique',"Unique"], 
              'S':  ["Unique", "Corrupted"], 
              'G':  ["Unique", "Corrupted"]
              }

In [91]:
df = pd.DataFrame()
flist = os.listdir(sprite_path)
df['raw'] = flist
df = df.raw.str.extract('^(?P<unitid>\d+)(?P<unitname>[A-Z][a-z]+)(?P<labels>[A-Z]+)?_(?P<classid>\d+)(?P<classname>[A-Za-z]+)_(?P<weapon>\w+)').fillna('')
df['raw'] = flist
df = df.loc[df.labels != 'LC']
df[['gender', 'faction']] = df.apply(lambda x: label_map[x.labels],axis=1, result_type='expand')
df.loc[(df.unitname == 'Morph') & (df.labels == 'S'), ['unitname']] = 'SilverCorrupted'
df.loc[(df.unitname == 'Morph') & (df.labels == 'G'), ['unitname']] = 'GoldCorrupted'
df.loc[(df.unitname == 'Morph') & (df.labels == ''), ['unitname', 'gender', 'faction']] = 'Corrupted', 'Monster', 'Corrupted'
df['engaged'] = df.labels.apply(lambda x: True if x == 'E' else False)
wp_remap = {'Ax': 'Axe', "Scroll": "Art", "Knife":"Dagger", "MagicBook":"Tome"}
df.weapon = df.weapon.map(lambda x: wp_remap[x] if x in wp_remap else x)
df.classname = df.classname.replace(regex='(\w+)(?<=[a-z])([A-Z])(\w+)', value='\g<1> \g<2>\g<3>')
df.classname = df.classname.str.replace('Ax ', 'Axe ')
class_remap = {
    '(Sword|Axe|Lance) Knight': '\g<1> Cavalier',
    'Dragon Knight': 'Wyvern Knight',
    'Griffon Knight': 'Griffin Knight',
    '^Monk': 'Martial Monk',
    'Master Monk': 'Martial Master',
    'Brave Hero': 'Hero',
    '(Sword|Axe|Lance) Pegasus': '\g<1> Flier',
    'Sword Master': 'Swordmaster'
}
df.classname = df.classname.replace(regex=class_remap)

fix = df.loc[(df.weapon =='NoWeapon') & (df.classname.str.contains('Sword (?:Armor|Cavalier|Flier)'))].copy()
fix1 = fix.copy()
fix1.classname = fix1.classname.str.replace('Sword', 'Axe')
fix2 = fix.copy()
fix2.classname = fix2.classname.str.replace('Sword', 'Lance')
df = pd.concat([df, fix1, fix2])

def pathing(x):
    path = [out_path, x.faction, x.unitname]
    if x.classname != x.unitname: path.append(x.classname)
    fname = f"{x.gender+'-' if x.gender != 'Unique' else ''}"
    if x.labels == 'E': fname+= f'{x.classname}-Engaged.png'
    else: fname+= f'{x.classname}-{x.weapon}.png'
    os.makedirs(os.path.join(*path), exist_ok=True)
    newpath = os.path.join(*path,fname)
    return fname, newpath
df[['fname', 'path']] = df.apply(pathing,axis=1, result_type='expand')

df.labels.unique()
df.loc[df.unitname == 'Anna'].sort_values(['engaged', 'classname', 'weapon'])
# len(df.loc[df.classname.str.contains('Monk')].sort_values(['engaged', 'classname', 'weapon']))

,unitid,unitname,labels,classid,classname,weapon,raw,gender,faction,engaged,fname,path
3901,552,Anna,,624,Archer,Bow,552Anna_624Archer_Bow.png,Unique,Unique,False,Archer-Bow.png,images/Unit Sprites/Engage/Unique/Anna/Archer/...
3909,552,Anna,,624,Archer,NoWeapon,552Anna_624Archer_NoWeapon.png,Unique,Unique,False,Archer-NoWeapon.png,images/Unit Sprites/Engage/Unique/Anna/Archer/...
3637,552,Anna,,631,Axe Armor,Axe,552Anna_631AxArmor_Ax.png,Unique,Unique,False,Axe Armor-Axe.png,images/Unit Sprites/Engage/Unique/Anna/Axe Arm...
5488,552,Anna,,631,Axe Armor,NoWeapon,552Anna_631SwordArmor_NoWeapon.png,Unique,Unique,False,Axe Armor-NoWeapon.png,images/Unit Sprites/Engage/Unique/Anna/Axe Arm...
4465,552,Anna,,638,Axe Cavalier,Axe,552Anna_638AxKnight_Ax.png,Unique,Unique,False,Axe Cavalier-Axe.png,images/Unit Sprites/Engage/Unique/Anna/Axe Cav...
4416,552,Anna,,638,Axe Cavalier,NoWeapon,552Anna_638SwordKnight_NoWeapon.png,Unique,Unique,False,Axe Cavalier-NoWeapon.png,images/Unit Sprites/Engage/Unique/Anna/Axe Cav...
4303,552,Anna,,618,Axe Fighter,Axe,552Anna_618AxFighter_Ax.png,Unique,Unique,False,Axe Fighter-Axe.png,images/Unit Sprites/Engage/Unique/Anna/Axe Fig...
2963,552,Anna,,618,Axe Fighter,NoWeapon,552Anna_618AxFighter_NoWeapon.png,Unique,Unique,False,Axe Fighter-NoWeapon.png,images/Unit Sprites/Engage/Unique/Anna/Axe Fig...
3000,552,Anna,,646,Axe Flier,Axe,552Anna_646AxPegasus_Ax.png,Unique,Unique,False,Axe Flier-Axe.png,images/Unit Sprites/Engage/Unique/Anna/Axe Fli...
5500,552,Anna,,646,Axe Flier,NoWeapon,552Anna_646SwordPegasus_NoWeapon.png,Unique,Unique,False,Axe Flier-NoWeapon.png,images/Unit Sprites/Engage/Unique/Anna/Axe Fli...


In [93]:
for i, x in df.iterrows():
    copy(sprite_path+'/'+x.raw, x.path)

In [116]:
# for x in range(0,7):
    # print(70+x*5, x*5, 70+x*5-(x*5))

df = pd.DataFrame()
df['base'] = [i*5+70 for i in range(0,7)]
df['values'] = [i*5 for i in range(0,7)]
df['F E D C B A S'.split()] = 0
for i, row in df.iterrows():
    row['F E D C B A S'.split()] =  [row.base - (i*5) for i in range(0,7)]
# df = df.set_index('base')
df

,base,values,F,E,D,C,B,A,S
0,70,0,70,65,60,55,50,45,40
1,75,5,75,70,65,60,55,50,45
2,80,10,80,75,70,65,60,55,50
3,85,15,85,80,75,70,65,60,55
4,90,20,90,85,80,75,70,65,60
5,95,25,95,90,85,80,75,70,65
6,100,30,100,95,90,85,80,75,70
